In [1]:
from Models import mobilenetv2, resnet, gatedmodel
import torch
import torchvision

# import torch and torch.nn
# convert torch to tflite
# get client
m_i_client, _ = mobilenetv2.mobilenetv2_splitter(num_classes=1000, weight_root="./Weights/" + "imagenet" + "/")
r_i_client, _  = resnet.resnet_splitter(num_classes=1000, weight_root="./Weights/" + "imagenet" + "/", layers=50)

m_c_client, _  = mobilenetv2.mobilenetv2_splitter(num_classes=10, weight_root="./Weights/" + "cifar-10" + "/")
r_c_client, _  = resnet.resnet_splitter(num_classes=10, weight_root="./Weights/" + "cifar-10" + "/", layers=50)

m_ch = [2**x for x in range(0, 5)]
m_i_encoders =[]
for ch in m_ch:
    m_i_encoders.append(mobilenetv2.MobileNetV2_middle(middle=ch))
    m_i_encoders[-1].load_state_dict(torch.load("./Weights/"+ "imagenet"+ "/middle/"+ "mobile"+ "_"+ "imagenet"+ "_middle_"+ str(ch)+ ".pth"))
    m_i_encoders[-1] = m_i_encoders[-1].in_layer

r_ch = [2**x for x in range(0, 6)]
r_i_encoders = []
for ch in r_ch:
    r_i_encoders.append(resnet.resnet_middle(middle=ch))
    r_i_encoders[-1].load_state_dict(torch.load("./Weights/"+ "imagenet"+ "/middle/"+ "resnet"+ "_"+ "imagenet"+ "_middle_"+ str(ch)+ ".pth"))
    r_i_encoders[-1] = r_i_encoders[-1].in_layer

m_c_encoders = []
for ch in m_ch:
    m_c_encoders.append(mobilenetv2.MobileNetV2_middle(middle=ch))
    m_c_encoders[-1].load_state_dict(torch.load("./Weights/"+ "cifar-10"+ "/middle/"+ "mobile"+ "_"+ "cifar-10"+ "_middle_"+ str(ch)+ ".pth"))
    m_c_encoders[-1] = m_c_encoders[-1].in_layer

r_c_encoders = []
for ch in r_ch:
    r_c_encoders.append(resnet.resnet_middle(middle=ch))
    r_c_encoders[-1].load_state_dict(torch.load("./Weights/"+ "cifar-10"+ "/middle/"+ "resnet"+ "_"+ "cifar-10"+ "_middle_"+ str(ch)+ ".pth"))
    r_c_encoders[-1] = r_c_encoders[-1].in_layer

m_i_gates = []
for ch in m_ch:
    m_i_gates.append(gatedmodel.ExitGate(in_planes=ch, height=112, width=112))
    m_i_gates[-1].load_state_dict(torch.load("./Weights/"+ "imagenet"+ "/gate/"+ "mobile"+ "_"+ "imagenet"+ "_gate_"+ str(ch)+ ".pth"))

r_i_gates = []
for ch in r_ch:
    r_i_gates.append(gatedmodel.ExitGate(in_planes=ch, height=56, width=56))
    r_i_gates[-1].load_state_dict(torch.load("./Weights/"+ "imagenet"+ "/gate/"+ "resnet"+ "_"+ "imagenet"+ "_gate_"+ str(ch)+ ".pth"))

m_c_gates = []
for ch in m_ch:
    m_c_gates.append(gatedmodel.ExitGate(in_planes=ch, height=16, width=16))
    m_c_gates[-1].load_state_dict(torch.load("./Weights/"+ "cifar-10"+ "/gate/"+ "mobile"+ "_"+ "cifar-10"+ "_gate_"+ str(ch)+ ".pth"))

r_c_gates = []
for ch in r_ch:
    r_c_gates.append(gatedmodel.ExitGate(in_planes=ch, height=8, width=8))
    r_c_gates[-1].load_state_dict(torch.load("./Weights/"+ "cifar-10"+ "/gate/"+ "resnet"+ "_"+ "cifar-10"+ "_gate_"+ str(ch)+ ".pth"))

# eval
m_i_client.eval()
r_i_client.eval()
m_c_client.eval()
r_c_client.eval()
for x in m_i_encoders:
    x.eval()
for x in r_i_encoders:
    x.eval()
for x in m_c_encoders:
    x.eval()
for x in r_c_encoders:
    x.eval()
for x in m_i_gates:
    x.eval()
for x in r_i_gates:
    x.eval()
for x in m_c_gates:
    x.eval()

# quant them
m_i_client = torch.ao.quantization.quantize_dynamic(m_i_client, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
r_i_client = torch.ao.quantization.quantize_dynamic(r_i_client, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
m_c_client = torch.ao.quantization.quantize_dynamic(m_c_client, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
r_c_client = torch.ao.quantization.quantize_dynamic(r_c_client, {torch.nn.Linear, torch.nn.Conv2d}, dtype=torch.qint8)
r_i_encoders = [torch.ao.quantization.quantize_dynamic(x, {torch.nn.Conv2d}, dtype=torch.qint8) for x in r_i_encoders]
m_i_encoders = [torch.ao.quantization.quantize_dynamic(x, {torch.nn.Conv2d}, dtype=torch.qint8) for x in m_i_encoders]
r_c_encoders = [torch.ao.quantization.quantize_dynamic(x, {torch.nn.Conv2d}, dtype=torch.qint8) for x in r_c_encoders]
m_c_encoders = [torch.ao.quantization.quantize_dynamic(x, {torch.nn.Conv2d}, dtype=torch.qint8) for x in m_c_encoders]
r_i_gates = [torch.ao.quantization.quantize_dynamic(x, {torch.nn.Conv2d}, dtype=torch.qint8) for x in r_i_gates]
m_i_gates = [torch.ao.quantization.quantize_dynamic(x, {torch.nn.Conv2d}, dtype=torch.qint8) for x in m_i_gates]
r_c_gates = [torch.ao.quantization.quantize_dynamic(x, {torch.nn.Conv2d}, dtype=torch.qint8) for x in r_c_gates]
m_c_gates = [torch.ao.quantization.quantize_dynamic(x, {torch.nn.Conv2d}, dtype=torch.qint8) for x in m_c_gates]

c_input = (torch.randn(1, 3, 32, 32))
i_input = (torch.randn(1, 3, 224, 224))


In [2]:
torch.onnx.export(m_i_client, i_input, "./Onnxmodel/m_i_client.onnx", opset_version=12, input_names=["input"], output_names=["output"])
torch.onnx.export(r_i_client, i_input, "./Onnxmodel/r_i_client.onnx", opset_version=12, input_names=["input"], output_names=["output"])
torch.onnx.export(m_c_client, c_input, "./Onnxmodel/m_c_client.onnx", opset_version=12, input_names=["input"], output_names=["output"])
torch.onnx.export(r_c_client, c_input, "./Onnxmodel/r_c_client.onnx", opset_version=12, input_names=["input"], output_names=["output"])

m_c_input = torch.randn(1, 32, 16, 16)
m_i_input = torch.randn(1, 32, 112, 112)
for i in range(len(m_ch)):
    m_c_g_input = torch.randn(1, m_ch[i], 16, 16)
    m_i_g_input = torch.randn(1, m_ch[i], 112, 112)
    torch.onnx.export(m_i_encoders[i], m_i_input, "./Onnxmodel/m_i_encoders_"+str(m_ch[i])+".onnx", opset_version=12, input_names=["input"], output_names=["output"])
    torch.onnx.export(m_i_gates[i], m_i_g_input, "./Onnxmodel/m_i_gates_"+str(m_ch[i])+".onnx", opset_version=12)
    torch.onnx.export(m_c_encoders[i], m_c_input, "./Onnxmodel/m_c_encoders_"+str(m_ch[i])+".onnx", opset_version=12, input_names=["input"], output_names=["output"])
    torch.onnx.export(m_c_gates[i], m_c_g_input, "./Onnxmodel/m_c_gates_"+str(m_ch[i])+".onnx", opset_version=12, input_names=["input"], output_names=["output"])

r_c_input = torch.randn(1, 64, 8, 8)
r_i_input = torch.randn(1, 64, 56, 56)
for i in range(len(r_ch)):
    r_c_g_input = torch.randn(1, r_ch[i], 8, 8)
    r_i_g_input = torch.randn(1, r_ch[i], 56, 56)
    torch.onnx.export(r_i_encoders[i], r_i_input, "./Onnxmodel/r_i_encoders_"+str(r_ch[i])+".onnx", opset_version=12, input_names=["input"], output_names=["output"])
    torch.onnx.export(r_i_gates[i], r_i_g_input, "./Onnxmodel/r_i_gates_"+str(r_ch[i])+".onnx", opset_version=12, input_names=["input"], output_names=["output"])
    torch.onnx.export(r_c_encoders[i], r_c_input, "./Onnxmodel/r_c_encoders_"+str(r_ch[i])+".onnx", opset_version=12, input_names=["input"], output_names=["output"])
    torch.onnx.export(r_c_gates[i], r_c_g_input, "./Onnxmodel/r_c_gates_"+str(r_ch[i])+".onnx", opset_version=12, input_names=["input"], output_names=["output"])

In [3]:
import onnx

m_i_client_onnx = onnx.load("./Onnxmodel/m_i_client.onnx")
r_i_client_onnx = onnx.load("./Onnxmodel/r_i_client.onnx")
m_c_client_onnx = onnx.load("./Onnxmodel/m_c_client.onnx")
r_c_client_onnx = onnx.load("./Onnxmodel/r_c_client.onnx")

m_i_encoders_onnx = [onnx.load("./Onnxmodel/m_i_encoders_"+str(m_ch[i])+".onnx") for i in range(len(m_ch))]
r_i_encoders_onnx = [onnx.load("./Onnxmodel/r_i_encoders_"+str(r_ch[i])+".onnx") for i in range(len(r_ch))]
m_c_encoders_onnx = [onnx.load("./Onnxmodel/m_c_encoders_"+str(m_ch[i])+".onnx") for i in range(len(m_ch))]
r_c_encoders_onnx = [onnx.load("./Onnxmodel/r_c_encoders_"+str(r_ch[i])+".onnx") for i in range(len(r_ch))]

m_i_gates_onnx = [onnx.load("./Onnxmodel/m_i_gates_"+str(m_ch[i])+".onnx") for i in range(len(m_ch))]
r_i_gates_onnx = [onnx.load("./Onnxmodel/r_i_gates_"+str(r_ch[i])+".onnx") for i in range(len(r_ch))]
m_c_gates_onnx = [onnx.load("./Onnxmodel/m_c_gates_"+str(m_ch[i])+".onnx") for i in range(len(m_ch))]
r_c_gates_onnx = [onnx.load("./Onnxmodel/r_c_gates_"+str(r_ch[i])+".onnx") for i in range(len(r_ch))]

# from onnx_tf.backend import prepare
# m_i_client_onnx_tf_prep = prepare(m_i_client_onnx)
# r_i_client_onnx_tf_prep = prepare(r_i_client_onnx)
# m_c_client_onnx_tf_prep = prepare(m_c_client_onnx)
# r_c_client_onnx_tf_prep = prepare(r_c_client_onnx)

# m_i_encoders_onnx_tf_prep = [prepare(x) for x in m_i_encoders_onnx]
# r_i_encoders_onnx_tf_prep = [prepare(x) for x in r_i_encoders_onnx]
# m_c_encoders_onnx_tf_prep = [prepare(x) for x in m_c_encoders_onnx]
# r_c_encoders_onnx_tf_prep = [prepare(x) for x in r_c_encoders_onnx]

# m_i_gates_onnx_tf_prep = [prepare(x) for x in m_i_gates_onnx]
# r_i_gates_onnx_tf_prep = [prepare(x) for x in r_i_gates_onnx]
# m_c_gates_onnx_tf_prep = [prepare(x) for x in m_c_gates_onnx]
# r_c_gates_onnx_tf_prep = [prepare(x) for x in r_c_gates_onnx]

# m_i_client_onnx_tf_prep.export_graph("./Tflitemodel/m_i_client.pb")
# r_i_client_onnx_tf_prep.export_graph("./Tflitemodel/r_i_client.pb")
# m_c_client_onnx_tf_prep.export_graph("./Tflitemodel/m_c_client.pb")
# r_c_client_onnx_tf_prep.export_graph("./Tflitemodel/r_c_client.pb")

# for i in range(len(m_ch)):
#     m_i_encoders_onnx_tf_prep[i].export_graph("./Tflitemodel/m_i_encoders_"+str(m_ch[i])+".pb")
#     m_i_gates_onnx_tf_prep[i].export_graph("./Tflitemodel/m_i_gates_"+str(m_ch[i])+".pb")
#     m_c_encoders_onnx_tf_prep[i].export_graph("./Tflitemodel/m_c_encoders_"+str(m_ch[i])+".pb")
#     m_c_gates_onnx_tf_prep[i].export_graph("./Tflitemodel/m_c_gates_"+str(m_ch[i])+".pb")

# for i in range(len(r_ch)):
#     r_i_encoders_onnx_tf_prep[i].export_graph("./Tflitemodel/r_i_encoders_"+str(r_ch[i])+".pb")
#     r_i_gates_onnx_tf_prep[i].export_graph("./Tflitemodel/r_i_gates_"+str(r_ch[i])+".pb")
#     r_c_encoders_onnx_tf_prep[i].export_graph("./Tflitemodel/r_c_encoders_"+str(r_ch[i])+".pb")
#     r_c_gates_onnx_tf_prep[i].export_graph("./Tflitemodel/r_c_gates_"+str(r_ch[i])+".pb")


2024-10-09 17:01:13.157130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 17:01:13.167037: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-09 17:01:13.167057: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-09 17:01:13.175780: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-09 17:01:13.764284: W tensorflow/compiler/tf

ModuleNotFoundError: No module named 'tensorflow_addons'